# Automate sleep data retrieval and build a pipeline! 
Download sleep data from fitbit -> Parse csv & Insert data to Postgres -> Send alert to slack and Visualize! 

## my first customized xpath!
* protips, opend up Chrom Console, and search for the path to verify! 
* General rule of thumb: try to avoid absolute path, especially when the pages has many elements, things can change frequently! 
* Quick [tutorial](https://www.guru99.com/xpath-selenium.html)

Btw, div uses contain(text(), 'SPECIFIED_TEXT')  
Whereas button uses and text() = 'SPECIFIED_TEXT

In [1]:
!conda info --envs

# conda environments:
#
base                     C:\ProgramData\Miniconda3
py38_ds               *  C:\Users\narut\.conda\envs\py38_ds



In [4]:
import os 
from dotenv import load_dotenv

load_dotenv()
os.getenv('sleep_email')

ModuleNotFoundError: No module named 'dotenv'

In [1]:
login_page = "https://accounts.fitbit.com/login?targetUrl=https%3A%2F%2Fwww.fitbit.com%2Flogin%2Ftransferpage%3Fredirect%3Dhttps%25253A%25252F%25252Fwww.fitbit.com%25252Fsettings%25252Fdata%25252Fexport&lcl=en_MY"
email = os.getenv('sleep_email')
password = os.getenv('sleep_password')

# Use Selenium 
from datetime import datetime 
import time
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import chromedriver_binary 
from webdriver_manager.chrome import ChromeDriverManager


driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(login_page)

element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/main/div/div/div/div[3]/form/div[1]/div/input'))
    )

# gotta do this the selenium way, for example!
# wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Sign in']"))).click()


driver.find_element("xpath", '/html/body/div[2]/div/main/div/div/div/div[3]/form/div[1]/div/input').send_keys(email);
driver.find_element("xpath", '/html/body/div[2]/div/main/div/div/div/div[3]/form/div[2]/div/input').send_keys(password);
driver.find_element("xpath", '/html/body/div[2]/div/main/div/div/div/div[3]/form/div[5]/button').click();



# click on custom button 
custom_button = "//input[@id='ember64' and @value='CUSTOM']"
element = WebDriverWait(driver, 10).until(            
        EC.presence_of_element_located((By.XPATH, custom_button)))
driver.find_element(By.XPATH, custom_button).click()


start_date_xpath = "//input[@id='ember92' and @data-test-qa='data-export-start-date'] "
driver.find_element(By.XPATH, start_date_xpath).click()



date_today_xpath = f"//div[@class='dayContainer']//*[contains(text(), {datetime.now().day})]"
driver.find_element(By.XPATH, date_today_xpath).click()

time.sleep(5)
download_button = "//button[@class='button important download-button' and text() = 'Download']"
driver.find_element(By.XPATH, download_button).click()
time.sleep(10)
driver.close()



[WDM] - Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 7.72M/7.72M [00:02<00:00, 3.40MB/s]
/var/folders/4s/qf_5w4sd3_bc6pyrz5jp4hsc0000gn/T/ipykernel_77406/2540995970.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


# Parsing CSV data
Initial plan was to do the following: 
```
f'fitbit_export_{datetime.now().date()}'
datetime.now().date().strftime('%Y')
```
As I assumed that the csv file name that contains the date will be the date for the data being downloaded (for example if I download for data 2019-12-11 the csv should contain that date), but it is in fact today's date.   
Becuase of this I cannot test on 2022-06-01 (whether month and day will actually contain 0), thus I will just check for prefix and remove the csv file afterwards. 

** Today's date is 11.12.2022

In [2]:
import os

download_path = '/Users/jiong/Downloads'
files = os.listdir(download_path)

data_file = [x for x in files if x.startswith('fitbit')]

# if more than 1 fitbit csv file in download path, we identify the latest file 
if len(data_file) > 1: 
    temp_dict = {}
    for x in data_file: 
        temp_dict[x] = datetime.strptime(x.split('_')[2].split('.csv')[0], '%Y%m%d')
    

In [3]:
temp_dict[next(iter(temp_dict))]

datetime.datetime(2022, 11, 10, 0, 0)

In [4]:
temp_dict

{'fitbit_export_20221110.csv': datetime.datetime(2022, 11, 10, 0, 0),
 'fitbit_export_20221111.csv': datetime.datetime(2022, 11, 11, 0, 0),
 'fitbit_export_20221113.csv': datetime.datetime(2022, 11, 13, 0, 0)}

In [5]:
# to get the latest file, and check for latset date 

## intit var with first element's value. 
temp_date = temp_dict[next(iter(temp_dict))]
final_file = []
for file, date in temp_dict.items():    
    if date > temp_date: 
        temp_date = date
        final_file = file
        

In [11]:
# load csv 
import pandas as pd 
full_path = download_path+"/"+final_file
sleep_data = pd.read_csv(full_path, skiprows = 13, nrows=1)
sleep_data

,Start Time,End Time,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Minutes REM Sleep,Minutes Light Sleep,Minutes Deep Sleep
0,13-11-2022 1:12AM,13-11-2022 8:17AM,361,64,34,425,65,244,52


In [15]:
sleep_data['Start Time']

0    13-11-2022 1:12AM
Name: Start Time, dtype: object

In [76]:
# move processed file to a new dir.. 

import shutil 
move_to_done_path = f"{download_path}/done/{final_file}"
shutil.move(full_path, move_to_done_path)

'/Users/jiong/Downloads/done/fitbit_export_20221112.csv'